In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3

In [11]:
import numpy as np
import os

In [12]:
import os
import zipfile

# zip_ref = zipfile.ZipFile("./Data.zip", 'r')
# zip_ref.extractall("data/")
# zip_ref.close()

train_data_dir = ('data/train')
test_data_dir = ('data/test')
valid_data_dir = ('data/valid')

img_width, img_height = 224, 224
batch_size = 32
input_shape = (img_width, img_height, 3)

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for multi-label classification
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 673 images belonging to 19 classes.
Found 99 images belonging to 19 classes.
Found 189 images belonging to 19 classes.


In [ ]:
from tensorflow.keras.applications import InceptionV3
# Load the InceptionV3 model pre-trained on ImageNet
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='sigmoid')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=10  # You can start with a small number of epochs and increase as needed
)

# Unfreeze some layers of the base model for fine-tuning
for layer in base_model.layers[:249]:
    layer.trainable = False
for layer in base_model.layers[249:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training the model
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=100  # Additional epochs for fine-tuning
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test accuracy: {test_accuracy:.2f}")

Epoch 1/10
21/21 [==============================] - 8s 193ms/step - loss: 0.3566 - accuracy: 0.0905 - val_loss: 0.1801 - val_accuracy: 0.3063
Epoch 2/10
21/21 [==============================] - 2s 97ms/step - loss: 0.2360 - accuracy: 0.2090 - val_loss: 0.1532 - val_accuracy: 0.5250
Epoch 3/10
21/21 [==============================] - 2s 94ms/step - loss: 0.1968 - accuracy: 0.2980 - val_loss: 0.1363 - val_accuracy: 0.6062
Epoch 4/10
21/21 [==============================] - 2s 107ms/step - loss: 0.1695 - accuracy: 0.3729 - val_loss: 0.1297 - val_accuracy: 0.6313
Epoch 5/10
21/21 [==============================] - 2s 105ms/step - loss: 0.1474 - accuracy: 0.4727 - val_loss: 0.1093 - val_accuracy: 0.7125
Epoch 6/10
21/21 [==============================] - 2s 104ms/step - loss: 0.1359 - accuracy: 0.5429 - val_loss: 0.1107 - val_accuracy: 0.6875
Epoch 7/10
21/21 [==============================] - 2s 110ms/step - loss: 0.1358 - accuracy: 0.5507 - val_loss: 0.1147 - val_accuracy: 0.6875
Epoch 8/

In [ ]:
# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
#     MaxPooling2D((2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(train_generator.num_classes, activation='sigmoid')  # Use sigmoid for multi-label classification
# ])

# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


In [16]:
# epochs = 100
# model.fit(
#     train_generator,
#     steps_per_epoch=train_generator.samples // batch_size,
#     epochs=epochs,
#     validation_data=valid_generator,
#     validation_steps=valid_generator.samples // batch_size
# )

# # Save the model
model.save('multi_label_model.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test Accuracy: {test_acc}, Test Loss: {test_loss}')


In [29]:
# Load the model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
model = load_model('multi_label_model.h5')

# Function to preprocess new image
def preprocess_image(img_path, target_size):
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

# img_path = 'naskunxsate.jpg'
img_path = 'nasxsate.jpg'
processed_image = preprocess_image(img_path, target_size=(224, 224))
predictions = model.predict(processed_image)
threshold = 0.019
predicted_labels = (predictions > threshold).astype(int)
label_map = train_generator.class_indices
label_map = dict((v,k) for k,v in label_map.items())
predicted_classes = [label_map[i] for i in range(len(predicted_labels[0])) if predicted_labels[0][i] == 1]
print(f'Predicted labels for the image: {predicted_classes}')


1/1 [==============================] - 1s 1s/step
Predicted labels for the image: ['07. Nasi Kuning', '14. Sate Madura', '19. Nasi Putih']
